This is a example that obtains the tritium breeding ratio (TBR)
for a parametric submersion reactor and specified the faceting and merge
tolerance when creating the dagmc model.

In [ ]:
import matplotlib.pyplot as plt
import neutronics_material_maker as nmm
import openmc
import paramak
import openmc_dagmc_wrapper as odw

This section remakes the blankt material (FLiBe) for a range of temperatures and obtains the TBR for each one.

In [ ]:
def make_model_and_simulate(temperature):
    """Makes a neutronics Reactor model and simulates the flux

    Arguments:
        temperature: the temperature of the submersion blanket in Kelivin
    """

    # this can just be set as a string as temperature is needed for this
    # material
    flibe = nmm.Material.from_library(name='FLiBe', temperature=temperature, temperature_to_neutronics_code=False)

    source = openmc.Source()
    # sets the location of the source to x=0 y=0 z=0
    source.space = openmc.stats.Point((my_reactor.major_radius, 0, 0))
    # sets the direction to isotropic
    source.angle = openmc.stats.Isotropic()
    # sets the energy distribution to 100% 14MeV neutrons
    source.energy = openmc.stats.Discrete([14e6], [1])

    # makes the neutronics model from the geometry and material allocations
    neutronics_model = paramak_neutronics.NeutronicsModel(
        h5m_filename='dagmc.h5m',
        source=source,
        materials={
            'inboard_tf_coils_mat': 'eurofer',
            'center_column_shield_mat': 'eurofer',
            'divertor_mat': 'eurofer',
            'firstwall_mat': 'eurofer',
            'blanket_rear_wall_mat': 'eurofer',
            'blanket_mat': flibe,
            'supports_mat': 'eurofer'},
        cell_tallies=['TBR'],
    )

    # simulate the neutronics model
    h5m_filename = neutronics_model.simulate(
        simulation_batches=2,
        simulation_particles_per_batch=10,  # to get more accurate results this will need increasing
    ) 
    
    results = odw.process_results(statepoint_filename=h5m_filename)
    return results['TBR']['result']


# loops through different temperatures finding the TBR value at each one
tbr_values = []
temperature_values = [32, 100, 300, 500]
for temperature in temperature_values:
    tbr = make_model_and_simulate(temperature)
    tbr_values.append(tbr)

# plots the results
plt.scatter(temperature_values, tbr_values)
plt.xlabel('FLiBe Temperature (degrees C)')
plt.ylabel('TBR')
plt.show()